Neste exemplo você irá construir um classificador binário para o dataset do IMDB (NLP Problem)

O imdb dataset já vem com o Keras e já é pré-processado. Os reviews (sequência de palavras) já estão organizados em sequências de inteiros, em que cada inteiro significa uma palavra específica no dicionário.


In [1]:
import tensorflow as tf

As 10000 palavras mais frequentes serão mantidas no dataset

In [2]:
# call load_data with allow_pickle implicitly set to true
dataset = tf.keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = dataset.load_data(num_words=10000)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Train_data e test_data são as listas de reviews, cada review é uma lista de índices de palavras (texto do review codificado no índice de palavras)


In [3]:
print(train_data[6])

[1, 6740, 365, 1234, 5, 1156, 354, 11, 14, 5327, 6638, 7, 1016, 2, 5940, 356, 44, 4, 1349, 500, 746, 5, 200, 4, 4132, 11, 2, 9363, 1117, 1831, 7485, 5, 4831, 26, 6, 2, 4183, 17, 369, 37, 215, 1345, 143, 2, 5, 1838, 8, 1974, 15, 36, 119, 257, 85, 52, 486, 9, 6, 2, 8564, 63, 271, 6, 196, 96, 949, 4121, 4, 2, 7, 4, 2212, 2436, 819, 63, 47, 77, 7175, 180, 6, 227, 11, 94, 2494, 2, 13, 423, 4, 168, 7, 4, 22, 5, 89, 665, 71, 270, 56, 5, 13, 197, 12, 161, 5390, 99, 76, 23, 2, 7, 419, 665, 40, 91, 85, 108, 7, 4, 2084, 5, 4773, 81, 55, 52, 1901]


Train_labels e test_labels são as listas de 0’s e 1’s (0 = negativo review e 1 = positivo review)


In [4]:
print(train_labels[6])

1


Veja que nenhum índice de palavra irá exceder 10000

In [5]:
max([max(sequence) for sequence in train_data])

9999

Não podemos inserir um lista de inteiros na rede. Temos que converter a listar para um tensor no formato (samples, word_indices)

Por exemplo, transformar a sequência [3, 5] em um vetor de 10.000 dimensões que seria todos os 0s, exceto os índices 3 e 5, que seriam 1s.

In [8]:
import numpy as np

In [6]:
def vectorize_sequences(sequences, dimension=10000):
  # Creates an all-zero matrix of shape (len(sequences), dimension)
  results = np.zeros((len(sequences), dimension))

  for i, sequence in enumerate(sequences):
    # Sets specific indices of results[i] to 1s
    results[i, sequence] = 1.

  return results

In [9]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [10]:
print(x_train[0])
print(x_train.shape)

[0. 1. 1. ... 0. 0. 0.]
(25000, 10000)


O mesmo para os vetores de labels


In [12]:
y_train = np.asarray(train_labels).astype('float32')
print(y_train.shape, train_labels.shape)
y_test = np.asarray(test_labels).astype('float32')

(25000,) (25000,)


**Construindo a rede**

O input é um vetor e os labels são escalares (1’s e 0’s). Vamos usar um fully connected (Dense) com relu como activation function.

Dense(16, activation='relu’)

16: é o argumento para cada dense layer. Ou seja, é o número de hidden units da camada (é a dimensão da representação da camada). 


**Configurando a rede**

Iremos configurar a rede com duas camadas intermediárias com 16 unidades ocultas cada.
Uma terceira camada que produzirá a previsão escalar em relação ao sentimento do review em questão

In [13]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

c:\Users\mcoel\Documents\self_repos\Colab-AI-PucMinas\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Escolhendo a loss function e o optimizer (como strings por já fazerem parte do Keras)


In [ ]:
from tensorflow.keras import losses
from tensorflow.keras import metrics
# from keras.optimizers import SGD

# model.compile(optimizer=RMSprop(lr=0.001),
# loss=losses.binary_crossentropy,
# metrics=[metrics.binary_accuracy])

No entanto, você pode deixar tudo como padrão:

In [14]:
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])

Para monitorar o treinamento, criamos o conjunto de validação separando 10000 amostras do conjunto original

In [15]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

Iremos treinar o modelo por 20 épocas (20 iterações sobre todas as amostras do treinamento) em um batch de 512 amostras.


In [16]:
model.fit(partial_x_train,
partial_y_train,
epochs=100,
batch_size=512,
validation_data=(x_val, y_val))

Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - acc: 0.6842 - loss: 0.6024 - val_acc: 0.8373 - val_loss: 0.4206
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - acc: 0.8903 - loss: 0.3478 - val_acc: 0.8608 - val_loss: 0.3461
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - acc: 0.9174 - loss: 0.2523 - val_acc: 0.8818 - val_loss: 0.2973
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - acc: 0.9370 - loss: 0.1962 - val_acc: 0.8890 - val_loss: 0.2746
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - acc: 0.9490 - loss: 0.1605 - val_acc: 0.8860 - val_loss: 0.2822
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - acc: 0.9602 - loss: 0.1359 - val_acc: 0.8863 - val_loss: 0.2882
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - acc: 0.9682 - loss: 0.1108 - val_acc: 0.8856 - val_loss: 0.2950
Epoch 8/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - acc: 0.9698 - loss: 0.1045 - val_acc: 0.8858 - val_loss: 0.3112
Epoch 9/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - ac

**Usando uma rede treinada para gerar previsões sobre novos dados**

Gerando a probabilidade de cada análise ser positiva com o método predict

In [17]:
print(model.predict(x_test))

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
[[2.7128455e-05]
 [1.0000000e+00]
 [9.9985033e-01]
 ...
 [2.8226060e-07]
 [1.1168097e-05]
 [9.9983478e-01]]


A rede está confiante para algumas amostras (0,99 ou mais, ou 0,01 ou menos), mas menos confiante para outras (0,6, 0,4).

**Treinando o modelo do zero**

Observe que no treino abaixo não separamos o conjunto de treino em treino e validação, uma vez que não iremos validar o modelo durante o treinamento e sim após o mesmo com o model.evaluate()

In [ ]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])

In [ ]:
model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

In [ ]:
results = model.evaluate(x_test, y_test)

Referência: François Chollet. Deep Learning with Python. November 2017  